In [1]:
# Import OS module
import os

# Imports
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks, layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Notebook magic
%matplotlib inline

In [2]:
# Check if GPU available
!nvidia-smi

Mon Dec 04 08:12:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   39C    P8     4W /  N/A |    395MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Configure variables for Transfer learning
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)
grid_shape = (1, image_size, image_size, 3)

batch_size = 32

In [4]:
dataset_root = "D:/Chrome Downloads/Tomato/tomato"

train_dir = os.path.join(dataset_root, "train")
test_dir = os.path.join(dataset_root, "val")

In [5]:
# Define augmentations for train dataset and read the images
train_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0,
    # Filling for W/H shift
    fill_mode="nearest",
    # Width and Height shift
    width_shift_range=0.2,
    height_shift_range=0.2,
    # Random zooms
    zoom_range=0.2,
    # Random Shearing aug
    shear_range=0.2,
)

# Read data from directory
train_data = train_aug.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 10000 images belonging to 10 classes.


In [6]:
# Get the list of categories in training data
cats = list(train_data.class_indices.keys())

In [7]:
# Augmentations for test data
test_aug = ImageDataGenerator(
    # Rescale
    rescale=1/255.0
)

# Read data from directory
test_data = test_aug.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical"
)

Found 1000 images belonging to 10 classes.


In [8]:
# Load the base model
mbnet_v2 = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape
)

# Stop from being trainable
mbnet_v2.trainable = False

In [9]:
# Define the layers
inputs = keras.Input(shape=input_shape)

# Get the layer
x = mbnet_v2(inputs, training = False)

# Stack layers further
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(cats), activation="softmax")(x)

# Combine the model
model = Model(inputs=inputs, outputs=x)

# Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.0

In [10]:
# Compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define callbacks to use
early_stopping_cb = callbacks.EarlyStopping(monitor="loss", patience=3)

In [11]:
# Num epochs
epochs = 10

# Train model
history = model.fit(
    train_data,
    epochs=epochs,
    steps_per_epoch=218,
    callbacks=[early_stopping_cb]
)

Epoch 1/10
218/218 [==============================] - 301s 1s/step - loss: 1.2726 - accuracy: 0.5715
Epoch 2/10
218/218 [==============================] - 159s 727ms/step - loss: 0.7627 - accuracy: 0.7428
Epoch 3/10
218/218 [==============================] - 146s 668ms/step - loss: 0.6240 - accuracy: 0.7865
Epoch 4/10
218/218 [==============================] - 137s 630ms/step - loss: 0.5820 - accuracy: 0.8009
Epoch 5/10
218/218 [==============================] - 133s 610ms/step - loss: 0.5371 - accuracy: 0.8181
Epoch 6/10
218/218 [==============================] - 129s 593ms/step - loss: 0.4992 - accuracy: 0.8346
Epoch 7/10
218/218 [==============================] - 133s 608ms/step - loss: 0.4941 - accuracy: 0.8353
Epoch 8/10
218/218 [==============================] - 182s 837ms/step - loss: 0.4737 - accuracy: 0.8341
Epoch 9/10
218/218 [==============================] - 196s 899ms/step - loss: 0.4593 - accuracy: 0.8424
Epoch 10/10
218/218 [==============================] - 2946s 14s/st

In [12]:
model.evaluate(test_data)

32/32 [==============================] - 20s 614ms/step - loss: 0.5889 - accuracy: 0.7920


[0.588894248008728, 0.7919999957084656]